## Now we are gonna use the model

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [ ]:
## Loading the trained model, scaler pickle,onehot encoders
model=load_model('model.keras')

## load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo=pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [39]:
# Example input data
input_data = {
    'CreditScore': 900,
    'Geography': 'Spain',
    'Gender': 'Female',
    'Age': 44,
    'Tenure': 2,
    'Balance': 90000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 70000
}

In [40]:
# One-hot encode 'Geography'
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


c:\Users\Anumitha\miniconda\envs\ann\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [41]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,900,Spain,Female,44,2,90000,2,1,1,70000


In [42]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,900,Spain,0,44,2,90000,2,1,1,70000


In [44]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,900,0,44,2,90000,2,1,1,70000,0.0,0.0,1.0


In [45]:
# now data is scaler the data
input_scaled=scaler.transform(input_df)
input_scaled

array([[ 2.57733415, -1.09499335,  0.48508334, -1.04241787,  0.2225185 ,
         0.80843615,  0.64920267,  0.97481699, -0.5290988 , -0.99850112,
        -0.57946723,  1.73494238]])

In [46]:
# now data is ready..to predict
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 1s 982ms/step


array([[0.17785056]], dtype=float32)

In [47]:
prediction_proba = prediction[0][0]
prediction_proba 


0.17785056

In [48]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
